In [1]:
import os
os.chdir(r"\Users\Akhil\OneDrive\Desktop\animal-disease-classification-with-llm")

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_image_size: list
    params_is_augmentation: bool

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path
    

In [3]:
from classifier.constants import *
from classifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath: Path = CONFIG_FILE_PATH,
        params_filepath: Path = PARAMS_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([Path(self.config.artifacts_root)])
    
    # Callback configuratoin that can be used for training time
    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories(
            [
                Path(model_ckpt_dir),
                Path(config.tensorboard_root_log_dir)
            ]
        )
        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )
        return prepare_callback_config
    
    # Traiining configuration
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params

        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images")

        # ✅ create folder, not file
        create_directories([Path(training.trained_model_path).parent])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_image_size=params.IMAGE_SIZE,
            params_is_augmentation=params.AUGMENTATION,
        )
        return training_config


In [5]:
# You can get this from tensorflow documentation itself!
import time
class PrepareCallbacks:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{time.strftime('%Y%m%d-%H%M%S')}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )
    
    def get_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [6]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
from tensorflow import keras
import math

In [7]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        # ✅ load without optimizer state
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path,
            compile=False
        )

        # ✅ compile fresh optimizer
        self.model.compile(
            optimizer=tf.keras.optimizers.SGD(),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )
    
    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

        # ✅ Optional debug prints
        print("Train samples:", self.train_generator.samples)
        print("Valid samples:", self.valid_generator.samples)
        print("Classes:", self.train_generator.class_indices)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        path = Path(path)
        path.parent.mkdir(parents=True, exist_ok=True)  # ✅ only make parent dir
        model.save(path)


    def train(self, callback_list: list):

        self.steps_per_epoch = math.ceil(self.train_generator.samples / self.train_generator.batch_size)
        self.validation_steps = math.ceil(self.valid_generator.samples / self.valid_generator.batch_size)

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_data=self.valid_generator,
            validation_steps=self.validation_steps,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


In [8]:
import tensorflow as tf
print(tf.executing_eagerly())


True


In [9]:
tf.config.run_functions_eagerly(True)

In [10]:

try:
    config = ConfigurationManager()

    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallbacks(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)

    training.get_base_model()
    training.train_valid_generator()
    training.train(callback_list=callback_list)

except Exception as e:
    raise e

[2026-01-18 20:23:34,308 - INFO - common - yaml file: config\config.yaml loaded successfully]
[2026-01-18 20:23:34,311 - INFO - common - yaml file: params.yaml loaded successfully]
created directory at: artifacts
created directory at: artifacts\prepare_callbacks\checkpoint_dir
created directory at: artifacts\prepare_callbacks\tensorboard_log_dir
created directory at: artifacts\training
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.
Train samples: 312
Valid samples: 78
Classes: {'Coccidiosis': 0, 'Healthy': 1}


c:\Users\Akhil\Python\lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9s/step - accuracy: 0.4916 - loss: 14.7885[2026-01-18 20:26:50,452 - WARNING - saving_api - You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
20/20 ━━━━━━━━━━━━━━━━━━━━ 195s 10s/step - accuracy: 0.5000 - loss: 16.1241 - val_accuracy: 0.6026 - val_loss: 2.8848
Epoch 2/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9s/step - accuracy: 0.5533 - loss: 12.0902[2026-01-18 20:30:09,261 - WARNING - saving_api - You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
20/20 ━━━━━━━━━━━━━━━━━━━━ 199s 10s/step - accuracy: 0